This notebook will help you go through the steps for applying our model on the XRF elemental data. This way of classifying facies is in one section (i.e., one scanning unit). If you want to deal with multiple sections, you can adopt the idea to build loop or compine sections into one section file. Plenty of options can do the job. I don't develop a script to simplify each step because there are so many variants (e.g., input data type) out there. Better to stay raw and let others to customize. Otherwise, building a script coping with every variant should cost me too much time.

In [1]:
# needed packages and parameters
import pandas as pd
import numpy as np
from scipy.stats.mstats import gmean
from joblib import load

selected_elements = ['Si', 'S', 'Cl', 'K', 'Ca', 'Ti', 'Fe', 'Br', 'Rb', 'Sr', 'Zr', 'Ba']
window = 17

# Prepare data

## Read data
The elemental profiles will be extracted from the output file of Itrax XRF core scanner. It's better to be the reprocessed by Q-Spec.

In [2]:
result_df = pd.read_table('result.txt', skiprows=2)
result_df.head()

,filename,position (mm),sample surface,validity,E-gain,E-offset,F-slope,F-offset,cps,MSE,...,S2,W la,W lb1,W lb2,W La scat,W Lb1 scat,Cr inc,Cr coh,Dt,Unnamed: 54
0,C:\Data\WASA\N10\N10-1\XRF data\L000000.spe,6.0,6.20,1,0.019295,-0.016672,0.008555,0.095857,36846,2.08,...,0,484,408,96,261,204,4815,13380,0.155,NaN
1,C:\Data\WASA\N10\N10-1\XRF data\L000001.spe,8.0,6.21,1,0.019293,-0.016672,0.008363,0.095857,36893,1.98,...,0,456,421,48,298,199,4972,12890,0.154,NaN
2,C:\Data\WASA\N10\N10-1\XRF data\L000002.spe,10.0,6.22,1,0.019294,-0.016672,0.008340,0.095857,37097,2.00,...,0,550,450,104,247,263,4815,13049,0.156,NaN
3,C:\Data\WASA\N10\N10-1\XRF data\L000003.spe,12.0,6.23,1,0.019294,-0.016672,0.008208,0.095857,36880,1.98,...,0,492,491,0,258,200,4702,12995,0.155,NaN
4,C:\Data\WASA\N10\N10-1\XRF data\L000004.spe,14.0,6.25,1,0.019292,-0.016672,0.008175,0.095857,36926,2.22,...,0,478,467,136,168,112,4730,13696,0.154,NaN


## Centred log-ratio transformation
This step replaces the zero values to 1 and Centred log-ratio transorm the data.
x need to be a ndarray (matrix) of the 12 element intensities.

In [3]:
x = result_df.loc[:, selected_elements].values

x = np.where(x == 0, 1, x) 
x = np.log(x / gmean(x, axis = 1).reshape(x.shape[0], 1))

## Rolling

In [4]:
# build column names
new_cols = []
for fun in ['_mean', '_std']:
    new_cols = np.hstack(
        (new_cols, [col+fun for col in selected_elements])
        )

norm_df = pd.concat(
    [pd.DataFrame(x, index = result_df.index, columns = selected_elements), 
     result_df['position (mm)']], 
    join='inner', axis = 1)

# make sure the order is sorted by the composite_id and then 
# the order within section is sorted by the section depth simutaniously.
norm_df = norm_df.sort_values('position (mm)')

# start rolling
r_df = pd.DataFrame()

rolling = norm_df.loc[:, selected_elements].rolling(window = window, center = True)
r_df = r_df.append(
    pd.concat([rolling.mean(), rolling.std()], 
    axis = 1, join = 'inner'))

r_df.columns = new_cols
r_df = pd.concat(
    [r_df, norm_df['position (mm)']], 
    join = 'inner', axis = 1)

r_c_df = r_df.dropna(axis = 0)

In [5]:
r_c_df

,Si_mean,S_mean,Cl_mean,K_mean,Ca_mean,Ti_mean,Fe_mean,Br_mean,Rb_mean,Sr_mean,...,K_std,Ca_std,Ti_std,Fe_std,Br_std,Rb_std,Sr_std,Zr_std,Ba_std,position (mm)
8,0.100686,-1.767442,0.779980,1.750026,3.431247,1.324236,2.617090,-2.748810,-1.662726,-0.868306,...,0.063395,0.059612,0.073184,0.071208,0.376589,0.167767,0.077585,0.138379,0.273202,22.0
9,0.108582,-1.758384,0.771272,1.748334,3.430761,1.323342,2.610110,-2.770416,-1.649850,-0.878160,...,0.064165,0.059946,0.073823,0.072914,0.375014,0.170993,0.063639,0.142237,0.268240,24.0
10,0.109012,-1.745509,0.757928,1.745824,3.426191,1.320828,2.601935,-2.795100,-1.656402,-0.886605,...,0.062182,0.057865,0.072972,0.067083,0.387559,0.169640,0.057784,0.136936,0.300455,26.0
11,0.115515,-1.743977,0.746152,1.741467,3.422893,1.312879,2.592959,-2.785509,-1.661769,-0.892069,...,0.059711,0.056104,0.071183,0.067451,0.385667,0.170219,0.054286,0.142817,0.292741,28.0
12,0.127419,-1.740506,0.735562,1.742805,3.425264,1.313862,2.584608,-2.827074,-1.671790,-0.890342,...,0.059619,0.057408,0.071341,0.072699,0.414198,0.173044,0.055003,0.169436,0.310616,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,0.439888,-0.117487,1.195999,1.040438,4.199029,0.054949,2.286793,-3.059407,-2.340494,-0.138991,...,0.281297,0.638246,0.276023,0.250570,1.746980,0.762049,0.435229,0.325746,0.599129,1042.0
519,0.456335,-0.154308,1.185095,1.078160,4.201853,0.088951,2.271923,-3.116033,-2.297194,-0.169805,...,0.246518,0.633153,0.319537,0.221223,1.711724,0.772696,0.471611,0.319587,0.594819,1044.0
520,0.456902,-0.198115,1.169932,1.118913,4.199536,0.114933,2.280230,-3.178744,-2.214266,-0.202122,...,0.250849,0.637122,0.373180,0.233414,1.669536,0.741188,0.512929,0.318887,0.596177,1046.0
521,0.474151,-0.248401,1.141176,1.154694,4.165788,0.147120,2.279750,-3.251426,-2.073342,-0.233710,...,0.249419,0.660107,0.418037,0.232939,1.625755,0.494662,0.535289,0.311556,0.597899,1048.0


# Classify the facies

In [6]:
# the model is the best one in the latest development
model = load('r_roll_lr_model_20220120.joblib')
y_pred = model.predict(r_c_df.iloc[:, :-1])
y_pred

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 9,
       9, 9, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3,

In [7]:
facies_df = pd.read_csv('facies_info.csv')
facies_df

,Label,Epoch,Facies,Abbreviation,sea level,Remarks,Core sections
0,0,Holocene,high energy shallow marine,hsm,marine,"subtidal, coarse grained sediments e.g. shoref...",N13-4 36-61 (channel lag) // N13-4 2-36 (chann...
1,1,Holocene,high energy channel fill,hcf,NaN,subtidal,N41-4 1-98 // N41-2 70-97 // N40-1 2-100 (shal...
2,2,Holocene,low energy channel fill,lcf,NaN,subtidal,N31-1 1-94 // N41-2 2-60 // N32-3 2-95 // VVC0...
3,3,Holocene,sandflat,sf,NaN,intertidal environment,N40-4 2-100 // N36-4 2-100 // N26-4 2-90 // N3...
4,4,Holocene,mud flat,mf,NaN,intertidal environment,N11-1 22-35 // VVC20-2 60-118 // W5-2 22-75 //...
5,5,Holocene,lagoon,la,NaN,"""brackish"" environment / salt marsh",VVC17-3 95-119 // N71-4 2-42 // N43-1 15-115 /...
6,6,Holocene,peat,pt,terrestrial,including different peat types,VVC17-2 28-58 // VVC17-3 8-74 // N23-4 33-55 /...
7,7,Holocene,soil,so,terrestrial,developed on Pleistocene material,VVC17-2 60-66 // VVC17-2 66-76 (Ae horizon of ...
8,8,Pleistocene,intertidal/marine,pm,marine,marine sediments,N13-2 40-96 (unclear?) // N13-3 2-78 // N23-1 ...
9,9,Pleistocene,eolian/fluvial,pef,terrestrial,not differentiated / unclear,N15-1 2-90 // N15-2 2-90 // VVC17-1 2-107 // N...


In [8]:
# integers in y_pred will become strings once there is a successful replacement
# because the abbrevation is string
# hence, to make the condition continuously work, I change y_pred to strings first
a = y_pred.astype(str)

for index, abb in enumerate(facies_df.Abbreviation):
    #print(index, abb, a)
    a = np.where(a == str(index), abb, a)

a

array(['lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf',
       'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf',
       'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf',
       'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf',
       'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf',
       'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf', 'lcf',
       'lcf', 'hcf', 'hcf', 'hcf', 'hcf', 'hcf', 'hcf', 'hcf', 'hcf',
       'hcf', 'hcf', 'hcf', 'hcf', 'hcf', 'hcf', 'hcf', 'hsm', 'hsm',
       'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm',
       'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm',
       'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm',
       'hsm', 'hsm', 'hsm', 'hsm', 'hcf', 'hcf', 'hcf', 'hcf', 'hcf',
       'hcf', 'hcf', 'hcf', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm',
       'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm', 'hsm',
       'hsm', 'hsm',

In [9]:
out_df = pd.DataFrame({'position (mm)':r_c_df['position (mm)'].values, 'y_pred': a})

out_df

,position (mm),y_pred
0,22.0,lcf
1,24.0,lcf
2,26.0,lcf
3,28.0,lcf
4,30.0,lcf
...,...,...
510,1042.0,hsm
511,1044.0,hsm
512,1046.0,hsm
513,1048.0,hsm


Then you have a dataframe having position and the predicted facies labels (in abbreviation). You can export it in any type you prefer.

# Preparation of demo
This part is to prepare files and codes for people to use our model smoothly.

## Copy and modify facies info

In [7]:
recla_dir='../data/new facies types 20220120.xlsx'
excel_df = pd.read_excel(recla_dir, skiprows=5)
excel_df

,Label,Epoch,Facies,Abbreviation,sea level,Remarks,Core sections
0,0,Holocene,high energy shallow marine,hsm,marine,"subtidal, coarse grained sediments e.g. shoref...",N13-4 36-61 (channel lag) // N13-4 2-36 (chann...
1,1,Holocene,high energy channel fill,hcf,NaN,subtidal,N41-4 1-98 // N41-2 70-97 // N40-1 2-100 (shal...
2,2,Holocene,low energy channel fill,lcf,NaN,subtidal,N31-1 1-94 // N41-2 2-60 // N32-3 2-95 // VVC0...
3,3,Holocene,sandflat,sf,NaN,intertidal environment,N40-4 2-100 // N36-4 2-100 // N26-4 2-90 // N3...
4,4,Holocene,mud flat,mf,NaN,intertidal environment,N11-1 22-35 // VVC20-2 60-118 // W5-2 22-75 //...
5,5,Holocene,lagoon,la,NaN,"""brackish"" environment / salt marsh",VVC17-3 95-119 // N71-4 2-42 // N43-1 15-115 /...
6,6,Holocene,peat,pt,terrestrial,including different peat types,VVC17-2 28-58 // VVC17-3 8-74 // N23-4 33-55 /...
7,7,Holocene,soil,so,terrestrial,developed on Pleistocene material,VVC17-2 60-66 // VVC17-2 66-76 (Ae horizon of ...
8,8,Pleistocene,intertidal/marine,pm,marine,marine sediments,N13-2 40-96 (unclear?) // N13-3 2-78 // N23-1 ...
9,9,Pleistocene,eolian/fluvial,pef,terrestrial,not differentiated / unclear,N15-1 2-90 // N15-2 2-90 // VVC17-1 2-107 // N...


In [8]:
excel_df.to_csv('facies_info.csv', index=False)

## Decide how to transform label

In [55]:
# to enlarge the computation time difference we need a bigger size
y_pred = np.repeat(y_pred, 2000)

In [54]:
y_pred.shape

(10300,)

In [60]:
%%time
#1
y_facies = []

for y in y_pred:
    # the label is the same as the index
    # the 3rd column is Abbreviation
    y_facies.append(facies_df.iloc[y, 3])

y_facies = np.array(y_facies)

CPU times: user 21.5 s, sys: 5.64 ms, total: 21.5 s
Wall time: 21.5 s


In [61]:
%%time
#2
a = y_pred.astype(str)

for index, abb in enumerate(facies_df.Abbreviation):
    #print(index, abb, a)
    a = np.where(a == str(index), abb, a)

CPU times: user 832 ms, sys: 169 ms, total: 1 s
Wall time: 1.01 s


As I expected, the simple looping on every label (#1) takes longer time than the concise loop using the optimized funcion, np.where (#2). So I'll #2 in case there are big amount of data.